# Performance Comparison Notebook

This notebook compares the performance of different control methods for quadcopter control.

## Setup

First, let's import the necessary libraries:

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from quadcopter.dynamics import QuadState, Params
from quadcopter.simulation import simulate
from quadcopter.controllers import PIDController, PositionController, LQRController
from quadcopter.plotting import plot_3d_trajectory_comparison, plot_trajectory
from quadcopter.utils import create_pid_position_controller, create_lqr_controller

print("Libraries imported successfully")

## Define Test Scenario

Let's define a common test scenario for all controllers:

In [ ]:
# Test parameters
duration = 10.0
dt = 0.02
target_position = np.array([2.0, 1.0, 3.0])

# Initial state
initial_state = QuadState(
    pos=np.array([0.0, 0.0, 0.0]),
    vel=np.array([0.0, 0.0, 0.0]),
    quat=np.array([1.0, 0.0, 0.0, 0.0]),
    ang_vel=np.array([0.0, 0.0, 0.0])
)

print(f"Test scenario: Move from [0,0,0] to {target_position} in {duration} seconds")

## Controller 1: PID Control (Aggressive)

Let's implement an aggressive PID controller:

In [ ]:
# Aggressive PID controller
pid_controller_1 = create_pid_position_controller(
    target_pos=target_position,
    kp=(4.0, 4.0, 6.0),
    ki=(0.2, 0.2, 0.3),
    kd=(1.0, 1.0, 2.0)
)

# Run simulation
t1, states1, controls1 = simulate(
    duration=duration,
    dt=dt,
    controller=pid_controller_1,
    initial_state=initial_state,
    method="rk4"
)

print(f"PID Controller 1 completed with {len(t1)} data points")
print(f"Final position: [{states1[-1, 0]:.3f}, {states1[-1, 1]:.3f}, {states1[-1, 2]:.3f}]")

## Controller 2: PID Control (Conservative)

Let's implement a conservative PID controller:

In [ ]:
# Conservative PID controller
pid_controller_2 = create_pid_position_controller(
    target_pos=target_position,
    kp=(1.0, 1.0, 2.0),
    ki=(0.05, 0.05, 0.1),
    kd=(0.2, 0.2, 0.5)
)

# Run simulation
t2, states2, controls2 = simulate(
    duration=duration,
    dt=dt,
    controller=pid_controller_2,
    initial_state=initial_state,
    method="rk4"
)

print(f"PID Controller 2 completed with {len(t2)} data points")
print(f"Final position: [{states2[-1, 0]:.3f}, {states2[-1, 1]:.3f}, {states2[-1, 2]:.3f}]")

## Controller 3: LQR Control

Let's implement an LQR controller:

In [ ]:
# LQR controller
lqr_controller = create_lqr_controller()

# Run simulation
t3, states3, controls3 = simulate(
    duration=duration,
    dt=dt,
    controller=lqr_controller,
    initial_state=initial_state,
    method="rk4"
)

print(f"LQR Controller completed with {len(t3)} data points")
print(f"Final position: [{states3[-1, 0]:.3f}, {states3[-1, 1]:.3f}, {states3[-1, 2]:.3f}]")

## Controller 4: Hover Control (Baseline)

Let's implement a simple hover controller as baseline:

In [ ]:
from quadcopter.utils import create_hover_controller

# Hover controller
hover_controller = create_hover_controller()

# Run simulation
t4, states4, controls4 = simulate(
    duration=duration,
    dt=dt,
    controller=hover_controller,
    initial_state=initial_state,
    method="rk4"
)

print(f"Hover Controller completed with {len(t4)} data points")
print(f"Final position: [{states4[-1, 0]:.3f}, {states4[-1, 1]:.3f}, {states4[-1, 2]:.3f}]")

## Trajectory Comparison

Let's compare the trajectories of all controllers:

In [ ]:
# Compare trajectories
trajectories = [
    (states1, "Aggressive PID"),
    (states2, "Conservative PID"),
    (states3, "LQR"),
    (states4, "Hover (Baseline)")
]

plot_3d_trajectory_comparison(trajectories)

## Performance Metrics Calculation

Let's calculate performance metrics for each controller:

In [ ]:
def calculate_performance_metrics(t, states, target_pos):
    """Calculate performance metrics for a trajectory."""
    positions = states[:, 0:3]
    
    # Position error
    position_errors = positions - target_pos
    position_error_norms = np.linalg.norm(position_errors, axis=1)
    
    # Metrics
    final_error = np.linalg.norm(positions[-1] - target_pos)
    rmse = np.sqrt(np.mean(position_error_norms**2))
    mean_abs_error = np.mean(position_error_norms)
    max_error = np.max(position_error_norms)
    
    # Convergence time (time to reach 5% of final error)
    convergence_threshold = final_error * 1.05  # 5% tolerance
    converged_indices = np.where(position_error_norms <= convergence_threshold)[0]
    if len(converged_indices) > 0:
        convergence_time = t[converged_indices[0]]
    else:
        convergence_time = t[-1]
    
    return {
        'final_error': final_error,
        'rmse': rmse,
        'mean_abs_error': mean_abs_error,
        'max_error': max_error,
        'convergence_time': convergence_time
    }

# Calculate metrics for each controller
metrics1 = calculate_performance_metrics(t1, states1, target_position)
metrics2 = calculate_performance_metrics(t2, states2, target_position)
metrics3 = calculate_performance_metrics(t3, states3, target_position)
metrics4 = calculate_performance_metrics(t4, states4, target_position)

print("Performance Metrics:")
print("="*50)
print(f"{'Controller':<20} {'Final Error':<12} {'RMSE':<10} {'MAE':<10} {'Max Error':<12} {'Conv. Time':<12}")
print("-"*50)
print(f"{'Aggressive PID':<20} {metrics1['final_error']:<12.4f} {metrics1['rmse']:<10.4f} {metrics1['mean_abs_error']:<10.4f} {metrics1['max_error']:<12.4f} {metrics1['convergence_time']:<12.2f}")
print(f"{'Conservative PID':<20} {metrics2['final_error']:<12.4f} {metrics2['rmse']:<10.4f} {metrics2['mean_abs_error']:<10.4f} {metrics2['max_error']:<12.4f} {metrics2['convergence_time']:<12.2f}")
print(f"{'LQR':<20} {metrics3['final_error']:<12.4f} {metrics3['rmse']:<10.4f} {metrics3['mean_abs_error']:<10.4f} {metrics3['max_error']:<12.4f} {metrics3['convergence_time']:<12.2f}")
print(f"{'Hover (Baseline)':<20} {metrics4['final_error']:<12.4f} {metrics4['rmse']:<10.4f} {metrics4['mean_abs_error']:<10.4f} {metrics4['max_error']:<12.4f} {metrics4['convergence_time']:<12.2f}")

## Control Effort Analysis

Let's analyze the control effort for each controller:

In [ ]:
# Calculate control effort metrics
def calculate_control_effort(controls):
    """Calculate control effort metrics."""
    # Total control effort (sum of squares)
    total_effort = np.sum(controls**2)
    
    # Mean control effort
    mean_effort = np.mean(controls**2)
    
    # Max control effort
    max_effort = np.max(controls)
    
    return {
        'total': total_effort,
        'mean': mean_effort,
        'max': max_effort
    }

# Calculate control effort for each controller
effort1 = calculate_control_effort(controls1)
effort2 = calculate_control_effort(controls2)
effort3 = calculate_control_effort(controls3)
effort4 = calculate_control_effort(controls4)

print("\nControl Effort Metrics:")
print("="*50)
print(f"{'Controller':<20} {'Total Effort':<15} {'Mean Effort':<15} {'Max Effort':<12}")
print("-"*50)
print(f"{'Aggressive PID':<20} {effort1['total']:<15.2f} {effort1['mean']:<15.2f} {effort1['max']:<12.2f}")
print(f"{'Conservative PID':<20} {effort2['total']:<15.2f} {effort2['mean']:<15.2f} {effort2['max']:<12.2f}")
print(f"{'LQR':<20} {effort3['total']:<15.2f} {effort3['mean']:<15.2f} {effort3['max']:<12.2f}")
print(f"{'Hover (Baseline)':<20} {effort4['total']:<15.2f} {effort4['mean']:<15.2f} {effort4['max']:<12.2f}")

## Detailed Trajectory Plots

Let's create detailed plots for each controller:

In [ ]:
# Plot trajectories for each controller
controllers = [
    (t1, states1, controls1, "Aggressive PID"),
    (t2, states2, controls2, "Conservative PID"),
    (t3, states3, controls3, "LQR"),
    (t4, states4, controls4, "Hover (Baseline)")
]

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
fig.suptitle('Position vs Time for Different Controllers', fontsize=16)

for i, (t, states, controls, name) in enumerate(controllers):
    row = i // 2
    col = i % 2
    
    ax = axes[row, col]
    positions = states[:, 0:3]
    
    ax.plot(t, positions[:, 0], label='X')
    ax.plot(t, positions[:, 1], label='Y')
    ax.plot(t, positions[:, 2], label='Z')
    
    # Add target position lines
    ax.axhline(y=target_position[0], color='r', linestyle='--', alpha=0.7, label='X target')
    ax.axhline(y=target_position[1], color='g', linestyle='--', alpha=0.7, label='Y target')
    ax.axhline(y=target_position[2], color='b', linestyle='--', alpha=0.7, label='Z target')
    
    ax.set_title(name)
    ax.set_xlabel('Time [s]')
    ax.set_ylabel('Position [m]')
    ax.legend()
    ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

## Error Convergence Analysis

Let's analyze how quickly each controller converges to the target:

In [ ]:
# Plot error convergence
plt.figure(figsize=(12, 8))

controllers_data = [
    (t1, states1, "Aggressive PID"),
    (t2, states2, "Conservative PID"),
    (t3, states3, "LQR"),
    (t4, states4, "Hover (Baseline)")
]

for t, states, name in controllers_data:
    positions = states[:, 0:3]
    errors = np.linalg.norm(positions - target_position, axis=1)
    plt.plot(t, errors, label=name, linewidth=2)

plt.xlabel('Time [s]')
plt.ylabel('Position Error [m]')
plt.title('Position Error Convergence')
plt.legend()
plt.grid(True, alpha=0.3)
plt.yscale('log')  # Log scale to better see convergence
plt.show()

## Summary

This notebook demonstrated:
1. How to implement different control methods (PID, LQR, Hover)
2. How to run comparative simulations
3. How to visualize trajectory comparisons
4. How to calculate performance metrics
5. How to analyze control effort
6. How to evaluate convergence characteristics

Key findings:
- Aggressive PID controllers typically converge faster but may have higher overshoot
- Conservative PID controllers are more stable but slower to converge
- LQR controllers provide optimal control but require accurate system models
- Hover controllers serve as a baseline for comparison

This type of comparison is useful for:
- Selecting the best controller for a specific application
- Tuning controller parameters
- Academic research comparing control methods
- Benchmarking new control algorithms